In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt, mpld3
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
import plotly.graph_objects as go

In [9]:
np.random.seed(12345)

In [10]:
# Download the latest SO dataset here: https://drive.google.com/open?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV
data_2019 = pd.read_csv("developer_survey_2019/survey_results_public.csv", low_memory=False)

In [11]:
to_drop = ["Respondent", "OpenSource", "CareerSat", "JobSat", "JobSeek", "ResumeUpdate", "SurveyLength", "SurveyEase",
    "WelcomeChange", "EntTeams", "ScreenName", "LastIn", "SO", "Blockchain", "WorkChallenge", "BetterLife", "OffOn", "Currency",
    "CompTotal", "CompFreq", "MainBranch", "PlatformDesireNextYear", "LanguageDesireNextYear", "DatabaseDesireNextYear", 
    "MiscTechDesireNextYear", "WebFrameDesireNextYear", "MgrMoney", "ITperson", "Age1stCode"]

In [23]:
def col_drop(df: pd.DataFrame, to_drop: list) -> pd.DataFrame:

    df_dropped = df.copy()

    for flag in to_drop:
        try:
            df_dropped.drop([x for x in df_dropped.columns if flag in x], axis=1, inplace=True)
        except:
            pass

    return df_dropped

In [24]:
def string_replace(s: str) -> int:
    try:
        s = float(s)
    except:
        s = -1000

    return s

In [46]:
def text_clean(text: str) -> str:
    text = str(text).replace(" ", "_").replace("-", "_").replace(
        ",", "_").replace(".", "").replace("+", "p").replace("#", "s").replace(
            "/", "_").replace("'", "").replace("ʼ", "").replace(
                "(", "_").replace(")", "_").replace("’", "").replace(
                    "__", "_").replace("__", "_").replace("“", "").replace(
                        "”", "").replace(":", "_").replace("&", "_").lower()

    banned = ["participated_in_", "_or", "_of", "_etc", "_employees", "taken_", "_african_descent"]

    for t in banned: text = text.replace(t, "")

    return text

In [26]:
def create_controls(df: pd.DataFrame, exclude: str) -> dict:

    controls = {}

    for col in df.columns:
        if col != exclude:
            controls[col] = {
                "omitted": text_clean(pd.Series([x for sub in list(
                    data_2019[col]
                    .apply(text_clean)
                    .apply(lambda x: str(x).split(";"))) for x in sub])
                    .value_counts()
                    .idxmax()), 
                "controls": list(set([x for sub in list(
                    data_2019[col]
                    .apply(text_clean)
                    .apply(lambda x: str(x).split(";"))) for x in sub]))}

    return controls

In [27]:
def design_matrix(df: pd.DataFrame, controls: dict) -> pd.DataFrame:
    dm = df.copy()
    
    for control in controls.keys():
        dm[control] = dm[control].apply(text_clean)

        if control in num_columns:
            for c in controls[control]["controls"]:
                dm[control+"_"+c] = (dm[control] == c) * 1

        else:
            for c in controls[control]["controls"]:
                dm[control+"_"+c] = dm[control].apply(lambda x: c in str(x).split(";")) * 1

        dm.drop(control, axis=1, inplace=True)
        dm.drop(control+"_"+controls[control]["omitted"], axis=1, inplace=True)
    
    # TODO: Rearrange columns alphabetically

    #dm.T.

    return dm

In [28]:
data_2019 = col_drop(data_2019, to_drop)

In [29]:
# Only consider those with income between $10,000 and $250,000
data_2019 = data_2019[(data_2019["ConvertedComp"] >= 10000) & (data_2019["ConvertedComp"] <= 250000)]
data_2019["ConvertedComp"] = np.log(data_2019["ConvertedComp"])
data_2019 = data_2019.rename(columns = {"ConvertedComp": "Income"})

# Only consider US respondents
data_2019 = data_2019[data_2019["Country"] == "United States"]
data_2019.drop(["Country"], axis=1, inplace=True)

# Only consider 18+ respondents
data_2019 = data_2019[data_2019["Age"] >= 21]

# Only consider respondents in the workforce
data_2019 = data_2019[data_2019["Employment"] != "Retired"]
data_2019 = data_2019[data_2019["Employment"] != "Not employed, and not looking for work"]

data_2019 = data_2019[data_2019["WorkWeekHrs"] >= 5]

# Only consider those with at least some education
data_2019 = data_2019[data_2019["EdLevel"] != "I never completed any formal education"]

data_2019 = data_2019.fillna("no_answer")

num_columns = ["Age", "YearsCode", "YearsCodePro", "WorkWeekHrs", "CodeRevHrs"]

# Convert numeric columns to int
for col in num_columns:
    data_2019[col] = data_2019[col].astype("int32", errors="ignore")

#data_2019["Age1stCode"].replace("Younger than 5 years", "4", inplace=True)
data_2019["YearsCode"].replace("Less than 1 year", "0", inplace=True)
data_2019["YearsCode"].replace("More than 50 years", "51", inplace=True)

for col in num_columns:
    data_2019[col] = data_2019[col].apply(string_replace)

# Exclude respondents who selected multiple gender, race, or sexual orientation
# options
data_2019 = data_2019[~data_2019["Gender"].str.contains(";")]
data_2019 = data_2019[~data_2019["Ethnicity"].str.contains(";")]
data_2019 = data_2019[~data_2019["Sexuality"].str.contains(";")]

# Reset index
data_2019 = data_2019.reset_index(drop=True)

In [30]:
num_labels = [
    ["no_answer", "21-25", "26-30", "31-35", "35-40", "41-45", "45-50", "51-55", "55-60", "61-65", "66-"],
    #["no_answer","-20", "21-25", "26-30", "31-35", "35-40", "41-45", "45-50", "51-55", "55-60", "61-65", "66-"],
    ["no_answer","0-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "41-"],
    ["no_answer","0-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "41-"],
    ["no_answer","-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", "81-"],
    ["no_answer","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11-15", "16-20", "21-"]]

num_buckets = [
    np.array([-1001,0,25,30,35,40,45,50,55,60,65,100]),
    #np.array([-1001,0,20,25,30,35,40,45,50,55,60,65,100]),
    np.array([-1001,0,5,10,15,20,25,30,35,40,100]),
    np.array([-1001,0,5,10,15,20,25,30,35,40,100]),
    np.array([-1001,0,10,20,30,40,50,60,70,80,200]),
    np.array([-1001,0,1,2,3,4,5,6,7,8,9,10,15,20,200])
]

In [31]:
for i, col in enumerate(num_columns):
    data_2019[col] = pd.cut(data_2019[col], num_buckets[i], labels=num_labels[i]).astype("str")

In [32]:
controls = create_controls(data_2019, "Income")

In [33]:
data_2019 = design_matrix(data_2019, controls)

In [34]:
est = sm.OLS(endog=data_2019["Income"].values, exog=data_2019.drop("Income", axis=1).assign(const=1)).fit()

In [35]:
est.summary()

1.090      0.276      -0.009       0.032
PlatformWorkedWith_ibm_cloud_watson                                                                              -0.0248      0.024     -1.020      0.308      -0.072       0.023
PlatformWorkedWith_docker                                                                                         0.0037      0.009      0.402      0.688      -0.014       0.022
WebFrameWorkedWith_express                                                                                       -0.0013      0.010     -0.133      0.894      -0.021       0.018
WebFrameWorkedWith_reactjs                                                                                        0.0374      0.008      4.520      0.000       0.021       0.054
WebFrameWorkedWith_aspnet                                                                                         0.0178      0.012      1.429      0.153      -0.007       0.042
WebFrameWorkedWith_flask                                                                                          0.0058      0.012      0.493      0.622      -0.017       0.029
WebFrameWorkedWith_laravel                                                                                       -0.0141      0.017     -0.810      0.418      -0.048       0.020
WebFrameWorkedWith_django                                                                                        -0.0337      0.012     -2.790      0.005      -0.057      -0.010
WebFrameWorkedWith_spring                                                                                        -0.0153      0.012     -1.232      0.218      -0.040       0.009
WebFrameWorkedWith_no_answer                                                                                      0.0266      0.011      2.474      0.013       0.006       0.048
WebFrameWorkedWith_angular_angularjs                                                                             -0.0092      0.008     -1.082      0.279      -0.026       0.007
WebFrameWorkedWith_ruby_on_rails                                                                                 -0.0122      0.016     -0.744      0.457      -0.044       0.020
WebFrameWorkedWith_drupal                                                                                        -0.0595      0.021     -2.846      0.004      -0.101      -0.019
WebFrameWorkedWith_other_s__                                                                                      0.0177      0.013      1.379      0.168      -0.007       0.043
WebFrameWorkedWith_vuejs                                                                                         -0.0242      0.010     -2.340      0.019      -0.044      -0.004
MiscTechWorkedWith_apache_spark                                                                                  -0.0074      0.019     -0.398      0.691      -0.044       0.029
MiscTechWorkedWith_no_answer                                                                                      0.0059      0.009      0.641      0.522      -0.012       0.024
MiscTechWorkedWith_cordova                                                                                       -0.0307      0.019     -1.595      0.111      -0.068       0.007
MiscTechWorkedWith_pandas                                                                                         0.0344      0.013      2.554      0.011       0.008       0.061
MiscTechWorkedWith_flutter                                                                                        0.0949      0.032      2.935      0.003       0.032       0.158
MiscTechWorkedWith_net                                                                                           -0.0082      0.014     -0.595      0.552      -0.035       0.019
MiscTechWorkedWith_cryengine                                                                                     -0.0424      0.109     -0.390      0.696      -0.256       0.171
MiscTechWorkedWith_net_core                

In [39]:
sm.regression.linear_model.RegressionResultsWrapper

statsmodels.regression.linear_model.RegressionResultsWrapper

In [40]:
def double_selection(df: pd.DataFrame, controls: dict, controls_adj: list, category: str) -> pd.DataFrame:

    X = df[controls_adj].copy()

    W = df.drop(controls_adj, axis=1).assign(const=1).copy()

    W = W.drop("Income", axis=1)

    Y = df["Income"].copy()

    clf = LassoCV(cv=5, max_iter=10000, selection="random", n_jobs=-1)

    sfm = SelectFromModel(clf)

    for i, X_j in enumerate(X.columns):
        if i==0:
            A = sfm.fit(W, X[X_j]).get_support()
        else:
            A = A | sfm.fit(W, X[X_j]).get_support()

    B = sfm.fit(W, Y).get_support()

    return pd.concat([W.T[A | B].T, X], axis=1)

In [114]:
def output_graph(df: pd.DataFrame, controls: dict, cov_selection: bool=False, category: str=None, category_plain: str=None, title: str=None):

    output_file("test.html")

    controls_adj = [category+"_"+control for control in controls[category]["controls"] if controls[category]["omitted"] != control]

    if cov_selection:
        exog = double_selection(df, controls, controls_adj, category).assign(const=1)
    else:
        exog = df.drop("Income", axis=1).assign(const=1)

    est_full = sm.OLS(endog=df["Income"], exog=exog).fit()
    
    est_simple = sm.OLS(endog=df["Income"], exog=df.drop("Income", axis=1)[controls_adj].assign(const=1)).fit()
    
    output_full = pd.concat([est_full.params, est_full.bse*1.96], axis=1).rename(columns={0: "coef_full", 1: "conf_95_full"}).filter(like=category+"_", axis=0)

    output_simple = pd.concat([est_simple.params, est_simple.bse*1.96], axis=1).rename(columns={0: "coef_simple", 1: "conf_95_simple"}).filter(like=category+"_", axis=0)
    
    output = pd.concat([output_full, output_simple], axis=1)
    
    output.index = [control for control in controls[category]["controls"] if controls[category]["omitted"] != control]

    output.sort_values(by="coef_full", ascending=False, inplace=True)

    output["explained"] = output["coef_simple"] - output["coef_full"]

    #output = np.e**(output) - 1

    output["upper_simple"] = np.e**(output["coef_simple"] + output["conf_95_simple"]) - 1
    output["upper_full"] = np.e**(output["coef_full"] + output["conf_95_full"]) - 1

    output["lower_simple"] = np.e**(output["coef_simple"] - output["conf_95_simple"]) - 1
    output["lower_full"] = np.e**(output["coef_full"] - output["conf_95_full"]) - 1

    output["coef_simple"] = np.e**output["coef_simple"] - 1
    output["coef_full"] = np.e**output["coef_full"] - 1

    for x, y  in zip(["upper_simple", "upper_full"], ["coef_simple", "coef_full"]): output[x] = output[x] - output[y]
    for x, y  in zip(["lower_simple", "lower_full"], ["coef_simple", "coef_full"]): output[x] = output[y] - output[x]

    output_index = list(output.index)

    my_range=np.array(list(range(1,len(output_index)+1)))

    output_index = [x[:16] for x in output.index]
    
    for i in range(0, len(output_index), 2):
       output_index[i] = "<br>" + output_index[i]

    fig = go.Figure(data=[
        go.Bar(name="Unadjusted", x=my_range, y=output["coef_simple"], marker_color="#3493d3", opacity=0.2, showlegend=False, width=min(0.28, 0.28*len(my_range)/4), hovertemplate=None),
        go.Bar(name="w/ Controls", x=my_range, y=output["coef_full"], marker_color="red", opacity=0.2, showlegend=False, width=min(0.28, 0.28*len(my_range)/4), hovertemplate=None),
        go.Scatter(name="Unadjusted", x=my_range-0.2, y=output["coef_simple"], mode="markers", marker_color="#3493d3", marker_size=20, 
            error_y=dict(type="data", symmetric=False, array=output["upper_simple"], arrayminus=output["lower_simple"], width=5, thickness=2, visible=True),
            hovertemplate=
            "%{text}",
            text=[name.replace("<br>", "") + ": {:,.1%}".format(coef) + ", 95% conf: [{:,.1%}".format(lower) + ", {:,.1%}]".format(upper) for name, coef, upper, lower in zip(output_index, output["coef_simple"], output["coef_simple"] + output["upper_simple"], output["coef_simple"] - output["lower_simple"])]),
        go.Scatter(name="w/ Controls", x=my_range+0.2, y=output["coef_full"], mode="markers", marker_color="red", marker_size=20,
            error_y=dict(type="data", symmetric=False, array=output["upper_full"], arrayminus=output["lower_full"], width=5, thickness=2, visible=True),
            hovertemplate=
            "%{text}",
            text=[name.replace("<br>", "") + ": {:,.1%}".format(coef) + ", 95% conf: [{:,.1%}".format(lower) + ", {:,.1%}]".format(upper) for name, coef, upper, lower in zip(output_index, output["coef_full"], output["coef_full"] + output["upper_full"], output["coef_full"] - output["lower_full"])])],
        layout=go.Layout(title=go.layout.Title(text=title)))

    fig.update_layout(
        barmode="group", 
        bargroupgap=0.1,
        xaxis = dict(
            tickmode = "array",
            tickvals = my_range,
            ticktext = output_index,
            tickangle = 0,
            ticks = "outside",
            showline=True,
            linewidth=0.5,
            linecolor="black"),
        yaxis = dict(
            tickformat = ".1%",
            ticks = "outside",
            title = "Pay Gap vs. " + controls[category]["omitted"][:16],
            showline=True,
            linewidth=0.5,
            linecolor="black"),
        width=max(500, len(my_range) * 100),
        plot_bgcolor="white",
        title_font = dict(size=20),
        legend=go.layout.Legend(
            x=0,
            y=1.13,
            orientation="h")
    )

    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=0.5,
            y0=0,
            x1=len(my_range)+0.5,
            y1=0,
            line=dict(
                color="grey",
                width=2),
        )
    )
    
    #import os

    #if not os.path.exists("images"):
    #    os.mkdir("images")

    #fig.write_image("images/test3.svg", scale=1)

    fig.show()
    
    return output, est_full

In [42]:
def prune_df(df: pd.DataFrame, category: str) -> pd.DataFrame:
    """
    Prunes dataframe columns to only include those with "category_" in the name
    
    Returns: Pruned dataframe
    """

    matching = [x for x in df.columns if category+"_" in x]

    return df[matching].copy()

In [59]:
def explain(df: pd.DataFrame, regression, controls: dict, cat_to_explain: str, coef_to_explain: str, top: int = 5):
    """
    Decomposes explained portion of income gap between groups and returns top explainers

    Decomposition follows methodology of (Gelbach 2016)

    Returns: Dataframe with top 3 explainers and respective explained gaps
    """

    name = cat_to_explain + "_" + coef_to_explain

    df_copy = df.copy()

    X_1 = prune_df(df_copy, cat_to_explain).assign(const=1)

    explainers = [control for control in controls.keys() if control != cat_to_explain]

    results = pd.DataFrame()
    
    for i, explainer in enumerate(explainers):

        X_2 = prune_df(df_copy, explainer)

        X_2 = X_2.reindex(sorted(X_2.columns), axis=1)

        B_2 = regression.params.filter(like=explainer+"_").sort_index()

        X_2 = X_2[B_2.index]

        H_k = X_2.values @ B_2

        if i == 0:
            results = pd.DataFrame(sm.OLS(endog=H_k, exog=X_1).fit().params.rename(explainer))
        else:
            d = sm.OLS(endog=H_k, exog=X_1).fit().params.rename(explainer)

            results = results.join(d)

    neg = results.T.filter(like=name).sort_values(by=name)[name].sum() < 0

    results = results.T.filter(like=name).sort_values(by=name, ascending=neg)

    results.columns = [coef_to_explain]

    return results.iloc[:top].append(results.iloc[top:].sum().rename("Other")), regression.params.filter(like=name).values[0]

In [101]:
def output_waterfall(df, coef_full):
    """
    Plots...


    """

    #df = np.e**(df) - 1
    #coef_full = np.e**(coef_full) - 1

    #df = df * (coef_full + 1)

    coef_simple = coef_full + df.values.flatten().sum()

    step_names = ["w/ Controls"] + list(df.index) + ["Unadjusted"]
    
    step_cum = [0]
    steps = np.array([coef_full] + list(df.values.flatten()) + [coef_simple])

    for i in range(len(step_names)):
        if i != 0:
            step_cum.append(sum(steps[:i]))
    
    step_min = np.array(step_cum)
    step_max = step_min + steps

    steps = np.e**np.array(steps) - 1

    my_range=np.array(list(range(1,len(steps)+1)))

    for i in range(0, len(step_names), 2):
       step_names[i] = "<br>" + step_names[i]

    fig = go.Figure(go.Waterfall(
        orientation="v",
        measure=["absolute"] + (["relative"] * len(df.index)) + ["total"],
        x=step_names,
        text=["{:,.1%}".format(steps[0])] + ["{:+,.1%}".format(step) for step in steps[1:-1]] + ["{:,.1%}".format(steps[-1])],
        textposition="outside",
        y=steps,
        connector = {"line":{"color":"grey"}},
        hovertemplate=
            "%{text}",
        name=output.columns[0],
        totals = {"marker":{"color":"#3493d3"}},
        decreasing = {"marker":{"color":"#ff4747"}}
    ))

    fig.update_layout(
        title = "Drivers of the " + output.columns[0][:16] + " pay gap", 
        xaxis = dict(
            tickmode = "array",
            tickvals = step_names,
            tickangle = 0,
            ticks = "outside",
            showline=True,
            linewidth=0.5,
            linecolor="black"),
        yaxis = dict(
            tickformat = ".1%",
            ticks = "outside",
            title = "Pay Gap",
            showline=True,
            linewidth=0.5,
            linecolor="black"),
        width=max(500, len(my_range) * 100),
        plot_bgcolor="white",
        title_font = dict(size=20)
    )

    fig.show()

In [105]:
test, est = output_graph(data_2019, controls, True, "Ethnicity", "race", "Pay gap vs. ")

In [115]:
test, est = output_graph(data_2019, controls, False, "Ethnicity", "race", "Pay gap vs. ")

In [99]:
output, coef_full = explain(data_2019, est, controls, "Ethnicity", "black_", 5)

output_waterfall(output, coef_full)

In [116]:
output, coef_full = explain(data_2019, est, controls, "Ethnicity", "black_", 5)

output_waterfall(output, coef_full)

In [68]:
print(np.sum(np.e**(output)-1, axis=0))
print(np.e**np.sum(output, axis=0)-1)

35_40    0.059041
dtype: float64
35_40    0.059287
dtype: float64


In [83]:
np.prod(np.e**output, axis=0) * (np.e**coef_full)

35_40    1.599684
dtype: float64

In [85]:
xs = [np.e**coef_full]
for i, x in enumerate(output.values):
    xs.append(xs[i]*np.e**x)

In [86]:
xs

[1.510152167554484,
 array([1.55605783]),
 array([1.59706374]),
 array([1.6211177]),
 array([1.63567828]),
 array([1.64940291]),
 array([1.59968386])]

In [87]:
xs = [coef_full]
for i, x in enumerate(output.values):
    xs.append(xs[i]+x)

In [88]:
xs

[0.41221041896444377,
 array([0.44215559]),
 array([0.46816678]),
 array([0.48311585]),
 array([0.49205757]),
 array([0.50041335]),
 array([0.46980602])]

In [93]:
np.e**np.array(xs) - 1

array([0.510152167554484, array([0.55605783]), array([0.59706374]),
       array([0.6211177]), array([0.63567828]), array([0.64940291]),
       array([0.59968386])], dtype=object)

In [558]:
controls

{'Hobbyist': {'omitted': 'yes', 'controls': ['no', 'yes']},
 'Employment': {'omitted': 'employed_full_time',
  'controls': ['employed_part_time',
   'no_answer',
   'employed_full_time',
   'independent_contractor_freelancer_or_self_employed']},
 'Student': {'omitted': 'no',
  'controls': ['no_answer', 'yes_full_time', 'no', 'yes_part_time']},
 'EdLevel': {'omitted': 'bachelors_degree_ba_bs_beng_etc_',
  'controls': ['secondary_school_eg_american_high_school_german_realschule_or_gymnasium_etc_',
   'no_answer',
   'bachelors_degree_ba_bs_beng_etc_',
   'some_college_university_study_without_earning_a_degree',
   'associate_degree',
   'other_doctoral_degree_phd_edd_etc_',
   'primary_elementary_school',
   'professional_degree_jd_md_etc_',
   'masters_degree_ma_ms_meng_mba_etc_']},
 'UndergradMajor': {'omitted': 'computer_science_computer_engineering_or_software_engineering',
  'controls': ['no_answer',
   'a_social_science_ex_anthropology_psychology_political_science_',
   'computer_s

In [573]:
test

,coef_full,conf_95_full,coef_simple,conf_95_simple,explained
no_answer,0.075432,0.051562,0.095888,0.071546,0.020456
participated_in_a_hackathon,0.040040,0.014063,0.102657,0.018730,0.062617
participated_in_online_coding_competitions_eg_hackerrank_codechef_topcoder_,0.022525,0.014609,0.016655,0.019832,-0.005870
contributed_to_open_source_software,0.011581,0.013683,0.123058,0.017172,0.111477
completed_an_industry_certification_program_eg_mcpd_,0.003938,0.016811,0.122910,0.022314,0.118972
taken_a_part_time_in_person_course_in_programming_or_software_development,0.002274,0.016822,0.077212,0.022664,0.074938
participated_in_a_full_time_developer_training_program_or_bootcamp,0.000662,0.018393,-0.007615,0.024166,-0.008276
received_on_the_job_training_in_software_development,-0.001761,0.012553,0.011001,0.016981,0.012762
taken_an_online_course_in_programming_or_software_development_eg_a_mooc_,-0.009758,0.012917,-0.022618,0.017464,-0.012860
